# Dynamic Optimization With Durable Goods and Bequest Taxation
In this project we elaborate on the two period consumer model presented in the lectures. 
Specifically, we examine the effect from introducind a durable good and a bequest tax. 



In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import ipywidgets as widgets
import time
from scipy import linalg
from scipy import optimize
import sympy as sm
from IPython.display import display

In [10]:
# define symbols
A = sm.symbols('A') # A is a Python variable representing the symbol A
Kt = sm.symbols('K_t')
Lt = sm.symbols('L_t')
alpha = sm.symbols('alpha')
wt = sm.symbols('w_t')
rt = sm.symbols('r_t')
kt = Kt/Lt

# define and show objective
pi = A*Kt**alpha*Lt**(1-alpha)-wt*Lt-rt*Kt
pi

A*K_t**alpha*L_t**(1 - alpha) - K_t*r_t - L_t*w_t

In [11]:
# First order conditions
dpi_dKt = sm.diff(pi,Kt)
dpi_dLt = sm.diff(pi,Lt)
dpi_dKt

A*K_t**alpha*L_t**(1 - alpha)*alpha/K_t - r_t

sample text to have something to commit